In [ ]:
# Author: Shubham Chaudhary
# Codes For: 5G Workshop 2023 @ IIIT Delhi

# Will be using 16-QAM for modulation and demodulation
# in all the simulations.

In [1]:
%config Completer.use_jedi=False
import numpy as np
import matplotlib.pyplot as plt

# **SISO Simulation With AWGN Channel**

In [72]:
# numSymbols = 10000                                                          # Number of symbols to transmit
# constellationMap = {"00":-3, "01":-1, "11":1, "10":3}                       # Used to map into constellation
# bitsPerSymbol = int(np.log2(16))                                            # bit per symbol = log2(M)

def _xor(bit1, bit2):
    """
    Function to perform XOR of two bits.
    Args:
        bit1 and bit2 (string): bits to perform XOR of
    Return:
        XOR of the bits (string)
    """
    return "0" if(bit1 == bit2) else "1"

def _binaryToGray(bits):
    """
    Function to convert binary bits into gray
    Args:
        bits (string): bits to convert
    Return:
        Gray encoded bits (string).
    """
    grayBits = bits[0]                                                      # Copies first bit as is
    for ii in range(1, len(bits)):
        grayBits += _xor(bits[ii-1], bits[ii])                              # XORing consecutive bits
    return grayBits

def decimalToGray(decimalNum, _constellationMap):
    """
    Function to convert decimal value of constellation into gray bits
    Args:
        decimalNum (int): decimal value
    Return:
        Gray bits (string)
    """
    return list(_constellationMap.keys())[list(_constellationMap.values()).index(decimalNum)]

def grayToDecimal(grayBits, _constellationMap):
    """
    Function to convert graybits of constellation into decimal number
    Args:
        grayBits (string): Gray bits
    Return:
        decimalNum (int): decimal value    
    """
    return _constellationMap[grayBits]

def generateSymbol(_numSymbols, _bitsPerSymbol):
    """
    Function to generate random sequence of bit to transmit
    Args:
        _numSymbol (int):  Number of symbols to transmit
        _bitsPerSymbol (int): Bits per symbol
    Return:
        Numpy Array (np.array): Array of dimension bitsPerSymbol x numOfSymbol with 4 bit symbols as string e.g. '1010'
    """    
    binaryInput = []
    binarySequence = np.random.randint(0, 2, (_bitsPerSymbol, _numSymbols)) # Generating random bits
    for jj in range(binarySequence.shape[1]):
        binaryInput.append(''.join(str(i) for i in binarySequence[:, jj]))  # Conveting them into 4bit symbols in type string
    return np.array(binaryInput)

def getSignal(inputSymbols, _constellation):
    """
    Function to modulate the given symbol
    Args:
        inputSymbol (string): input symbol as 4 bit binary symbol as string for 16 QAM 
    Return:
        signal (np.array): Array with complex signals
        realPartGray (np.array): Array with in-phase component of signal in gray of type string
        imagPartGray (np.array): Array with quadrature component of signal in gray of type string
    """
    realPartDec = []
    imagPartDec = []
    realPartGray, imagPartGray = [], []
    for i, symb in enumerate(inputSymbols):
        gray = _binaryToGray(symb)                                          # encoding input bits into gray for mapping onto constellation
        realPartGray.append(gray[:2])
        imagPartGray.append(gray[2:])                                       # first two bits correspond to real part and last two imaginary
        realPartDec.append(grayToDecimal(realPartGray[i], _constellation))
        imagPartDec.append(grayToDecimal(imagPartGray[i], _constellation))
        
    signal = 1/np.sqrt(10)*(np.array(realPartDec) + 1j*np.array(imagPartDec))   # sqrt(10) normalizes the symbol power
    return signal, np.array(realPartGray), np.array(imagPartGray)

def demodulateSignal(signal, _constellation):
    """
    Function to demodulate received signal using thresholding
    Args:
        signal (np.array): Array of complex signals
    Return:
        yRe_Gray (np.array): Array with real part of the demodulated signal in gray as type string
        yIm_Gray (np.array): Array with imaginary part of the demodulated signal in gray as type string
    """    
    yRe = np.sqrt(10)*np.real(signal)                                       # multiplying by sqrt(10) to denormalize the signal
    yIm = np.sqrt(10)*np.imag(signal)
    for ij in range(len(yRe)):
        # Thesholding to get the corresponding symbol on constellation
        # For yRe
        if yRe[ij] >= 2:
            yRe[ij] = 3
        elif yRe[ij] < 2 and yRe[ij] >= 0:
            yRe[ij] = 1
        elif yRe[ij] <= -2:
            yRe[ij] = -3
        elif yRe[ij] > -2 and yRe[ij] < 0:
            yRe[ij] = -1

        # For yIm
        if yIm[ij] >= 2:
            yIm[ij] = 3
        elif yIm[ij] < 2 and yIm[ij] >= 0:
            yIm[ij] = 1
        elif yIm[ij] <= -2:
            yIm[ij] = -3
        elif yIm[ij] > -2 and yIm[ij] < 0:
            yIm[ij] = -1

    # Mapping Decimal Values of signal into gray
    yRe_Gray = np.array([decimalToGray(x, _constellation) for x in yRe])
    yIm_Gray = np.array([decimalToGray(x, _constellation) for x in yIm])
    return yRe_Gray, yIm_Gray            

def getBER(yRe_Gray, yIm_Gray, xRe_Gray, xIm_Gray, numSymbols, bitsPerSymbol):
    """
    Function to calculate BER using gray bits send and received
    Args:
        yRe_Gray (string): gray real bits of demodulated signal
        yIm_Gray (string): gray imaginary bits of demodulated signal
        xRe_Gray (string): gray real bits of original signal
        xIm_Gray (string): gray imaginary bits of demodulated signal
        _numSymbol (int):  Number of symbols to transmit
        _bitsPerSymbol (int): Bits per symbol        
    Return:
        berError (float): ber of the signal
    """    
    bitError = (len((np.where(yRe_Gray!=xRe_Gray))[0]) + len((np.where(yIm_Gray!=xIm_Gray))[0]))
    return (bitError)/(numSymbols * bitsPerSymbol)       

def addAWGNChannel(signal, std):
    """
    Function to add AWGN noise to signal
    Args:
        signal (np.array): Array of complex symbols being transmitted
        std (float): noise variance
    Return:
        y (np.array): Array of complex received signal after adding noise
        n (np.array): Array with complex noise
    """      
    n = ((1/np.sqrt(2)) * np.random.normal(0, 1, size=len(signal))) + ((1/np.sqrt(2)) * 1j*np.random.normal(0, 1, size=len(signal)))
    y = signal + (np.multiply(n, 1/std))                                        # y = x + n
    return y, n

In [73]:
numSymbols = 10000                                                              # Number of symbols to transmit
constellationMap = {"00":-3, "01":-1, "11":1, "10":3}                           # Used to map into constellation
bitsPerSymbol = int(np.log2(16))                                                # bit per symbol = log2(M)

SNRdB = np.linspace(0, 20, 20)                                                  # Defining 20 SNRs in dB within the range 0 dB to 20 dB
SNR = np.array([10**(ii/10) for ii in SNRdB])                                   # Converting the SNRs into linear scale
std = np.sqrt(SNR)

# SISO with AWGN channel simuation for different SNRs
ber_awgn = []
for sd in std:
    symbs = generateSymbol(numSymbols, bitsPerSymbol)                           # Generating random sequence of bits
    signal, xRG, xIG = getSignal(symbs, constellationMap)                       # Generating complex signal from binary bits
    y, n = addAWGNChannel(signal, sd)                                           # Passing the signal through channel with only AWGN
    yR, yI = demodulateSignal(y, constellationMap)                              # Demodulating received signal                      
    ber_awgn.append(getBER(yR, yI, xRG, xIG, numSymbols, bitsPerSymbol))        # Calculating BER

In [75]:
# # Plotting
# plt.semilogy(SNRdB, ber_awgn, "-o", label="SISO AWGN")
# plt.grid(True)
# plt.xticks(fontsize=16)
# plt.yticks(fontsize=16)
# plt.xlabel("SNR in dB", fontsize=20)
# plt.ylabel("BER", fontsize=20)
# plt.title("16-QAM BER vs SNR Plot", fontsize=20)
# plt.legend(fontsize=16)

# **16 QAM modulation and demodulation**

In [2]:
class _16QAM():
    """
    Class to define 16-QAM. It has required function for generating signal 
    and its modulation adn demodulation including BER caluculation.
    """
    def __init__(self, numberOfSymbolsToTransmit=10000):
        self.numSymbols = numberOfSymbolsToTransmit
        self.constellationMap = {"00":-3, "01":-1, "11":1, "10":3}                      # Used to map onto constellation
        self.bitsPerSymbol = int(np.log2(16))

    def _xor(self, bit1, bit2):
        """
        Function to perform XOR of two bits.
        Args:
            bit1 and bit2 (string): bits to perform XOR of
        Return:
            XOR of the bits (string)
        """
        return "0" if(bit1 == bit2) else "1"

    def _binaryToGray(self, bits):
        """
        Function to convert binary bits into gray
        Args:
            bits (string): bits to convert
        Return:
            Gray encoded bits (string).
        """
        grayBits = bits[0]                                                              # Copies first bit as is
        for ii in range(1, len(bits)):
            grayBits += self._xor(bits[ii-1], bits[ii])                                 # XORing consecutive bits
        return grayBits

    def decimalToGray(self, decimalNum):
        """
        Function to convert decimal value of constellation into gray bits
        Args:
            decimalNum (int): decimal value
        Return:
            Gray bits (string)
        """
        return list(self.constellationMap.keys())[list(self.constellationMap.values()).index(decimalNum)]

    def grayToDecimal(self, grayBits):
        """
        Function to convert graybits of constellation into decimal number
        Args:
            grayBits (string): Gray bits
        Return:
            decimalNum (int): decimal value    
        """
        return self.constellationMap[grayBits]
    
    def generateSymbol(self):
        """
        Function to generate random sequence of bit to transmit
        Args:
            None
        Return:
            Numpy Array (np.array): Array of dimension bitsPerSymbol x numOfSymbol with 4 bit symbols as string e.g. '1010'
        """            
        binaryInput = []
        binarySequence = np.random.randint(0, 2, (self.bitsPerSymbol, self.numSymbols))
        for jj in range(binarySequence.shape[1]):
            binaryInput.append(''.join(str(i) for i in binarySequence[:, jj]))
        return np.array(binaryInput)

    def getSignal(self, inputSymbols):
        """
        Function to modulate the given symbol
        Args:
            inputSymbol (string): input symbol as 4 bit binary symbol as string for 16 QAM 
        Return:
            signal (np.array): Array with complex signals
            realPartGray (np.array): Array with in-phase component of signal in gray of type string
            imagPartGray (np.array): Array with quadrature component of signal in gray of type string
        """
        realPartDec = []
        imagPartDec = []
        realPartGray, imagPartGray = [], []
        for i, symb in enumerate(inputSymbols):
            gray = self._binaryToGray(symb)                                             # encoding input bits into gray
            realPartGray.append(gray[:2])
            imagPartGray.append(gray[2:])                                               # first two bits correspond to real part and last two imaginary
            realPartDec.append(self.grayToDecimal(realPartGray[i]))
            imagPartDec.append(self.grayToDecimal(imagPartGray[i]))
            
        signal = 1/np.sqrt(10)*(np.array(realPartDec) + 1j*np.array(imagPartDec))
        return signal, np.array(realPartGray), np.array(imagPartGray)          

    def demodulateSignal(self, signal):   
        """
        Function to demodulate received signal using thresholding
        Args:
            signal (np.array): Array of complex signals
        Return:
            yRe_Gray (np.array): Array with real part of the demodulated signal in gray as type string
            yIm_Gray (np.array): Array with imaginary part of the demodulated signal in gray as type string
        """             
        def threshold(y):
            "Function to threshold for getting actual symbol from recieved symbol"
            if y >= 2:
                return 3
            elif y < 2 and y >= 0:
                return 1
            elif y <= -2:
                return -3
            elif y > -2 and y < 0:
                return -1
        
        # Thesholding
        thresholdFunction = np.vectorize(threshold)                                     

        yRe = np.sqrt(10)*np.real(signal)
        yIm = np.sqrt(10)*np.imag(signal)
        yRe = thresholdFunction(yRe)                                                    # applying threshold frunction on all element of real part of signal array
        yIm = thresholdFunction(yIm)

        # Mapping Decimal Values of y into gray
        d2g = np.vectorize(self.decimalToGray)
        yRe_Gray = d2g(yRe)
        yIm_Gray = d2g(yIm)
        return yRe_Gray, yIm_Gray

    def getBER(self, yRe_Gray, yIm_Gray, xRe_Gray, xIm_Gray):
        """
        Function to calculate BER using gray bits send and received.
        Compares each 2 bit symbol (real or imag part og signal has 2 bit) to get bit error.
        Since gray bits are used only one bit flip can happend in 2 bits.
        Args:
            yRe_Gray (string): gray real bits of demodulated signal
            yIm_Gray (string): gray imaginary bits of demodulated signal
            xRe_Gray (string): gray real bits of original signal
            xIm_Gray (string): gray imaginary bits of demodulated signal
        Return:
            berError (float): ber of the signal
        """          
        bitError = (len((np.where(yRe_Gray!=xRe_Gray))[0]) + len((np.where(yIm_Gray!=xIm_Gray))[0]))
        return (bitError)/(self.numSymbols * self.bitsPerSymbol)                     


class AWGNChannel():
    """
    Modelling AWGN channel
    """
    def __init__(self, numOfSymbols):
        self.numOfSymbols = numOfSymbols

    def getChannel(self):
        return None, ((1/np.sqrt(2)) * np.random.normal(0, 1, size=self.numOfSymbols)) + ((1/np.sqrt(2)) * 1j*np.random.normal(0, 1, size=self.numOfSymbols))

    def addChannel(self, signal, std):
        h, n = self.getChannel()                                                        # h is None here
        y = signal + (np.multiply(n, 1/std))                                            # y = x + n
        return y, h, n


class RayleighChannel():
    """
    Modelling Rayleigh Channel with AWGN moise
    """
    def __init__(self, numOfSymbols):
        self.numOfSymbols = numOfSymbols

    def getChannel(self):
        h = ((1/np.sqrt(2)) * np.random.normal(0, 1, size=self.numOfSymbols)) + ((1/np.sqrt(2)) * 1j*np.random.normal(0, 1, size=self.numOfSymbols))
        n = ((1/np.sqrt(2)) * np.random.normal(0, 1, size=self.numOfSymbols)) + ((1/np.sqrt(2)) * 1j*np.random.normal(0, 1, size=self.numOfSymbols))
        return h, n

    def addChannel(self, signal, std):
        h, n = self.getChannel()
        y = (h*signal) + (np.multiply(n, 1/std))                                        # y = hx + n
        return y, h, n


class RXDiversityChannel():
    """
    Modelling SIMO Rayleigh Channel with AWGN moise
    """    
    def __init__(self, numOfSymbols, numReceiveAntenna=4):
        self.numOfSymbols = numOfSymbols
        self.numReceiveAntenna = numReceiveAntenna

    def getChannel(self):
        h = ((1/np.sqrt(2)) * np.random.normal(0, 1, size=(self.numOfSymbols, self.numReceiveAntenna, 1))) + ((1/np.sqrt(2)) * 1j*np.random.normal(0, 1, size=(self.numOfSymbols, self.numReceiveAntenna, 1)))
        n = ((1/np.sqrt(2)) * np.random.normal(0, 1, size=(self.numOfSymbols, self.numReceiveAntenna, 1))) + ((1/np.sqrt(2)) * 1j*np.random.normal(0, 1, size=(self.numOfSymbols, self.numReceiveAntenna, 1)))
        return h, n
    
    def addChannel(self, signal, std):
        h, n = self.getChannel()                                                        # h and n has shape numberOfSymbol x numReceiveAntenna x 1
        x = np.expand_dims(np.array([signal for i in range(self.numReceiveAntenna)]).T, axis=2) # making x a 3D matrix for consistency with h and n
        y = (h*x) + (np.multiply(n, 1/std))                                             # y = H.x + n
        return y, h, n
    

class MIMOChannel():
    """
    Modelling MIMO Rayleigh Channel with AWGN noise
    """
    def __init__(self, numOfSymbols, numTransmitAntenna=4, numReceiveAntenna=4):
        self.numOfSymbols = numOfSymbols//numTransmitAntenna
        self.numTransmitAntenna =  numTransmitAntenna
        self.numReceiveAntenna = numReceiveAntenna

    def getChannel(self):
        h = ((1/np.sqrt(2)) * np.random.normal(0, 1, size=(self.numOfSymbols, self.numReceiveAntenna, self.numTransmitAntenna))) + ((1/np.sqrt(2)) * 1j*np.random.normal(0, 1, size=(self.numOfSymbols, self.numReceiveAntenna, self.numTransmitAntenna)))
        n = ((1/np.sqrt(2)) * np.random.normal(0, 1, size=(self.numOfSymbols, self.numReceiveAntenna, 1))) + ((1/np.sqrt(2)) * 1j*np.random.normal(0, 1, size=(self.numOfSymbols, self.numReceiveAntenna, 1)))
        return h, n
    
    def addChannel(self, signal, std, technique="zf"):
        _signal = signal.reshape(self.numOfSymbols, self.numTransmitAntenna)
        _x = np.expand_dims(_signal, axis=2)
        h, n = self.getChannel()                                                            # h has shape numberOfSymbol x numReceiveAntenna x numTransmitAntenna
        if self.numReceiveAntenna == self.numTransmitAntenna:                               # MIMO spatial multiplexing possible with same number of antennas
            if technique == "svd":
                U, s, Vh = np.linalg.svd(h)                                                 # SVD, H = USVh
                S = np.expand_dims(s, axis=2)                                               # Making S 3D matrix    
                x = np.matmul(np.transpose(Vh.conj(), (0, 2, 1)), _x)                       # Precoding x with V = hermition(Vh)
                X = np.matmul(h, x)
                y = X + np.multiply(n, 1/std)                                               # y = H.x + n = USVhx + n

            elif technique == "zf":
                X = np.matmul(h, _x)
                y = X + np.multiply(n, 1/std)                                               # y = H.x + n
                U, S, Vh = None, None, None
        else:
            raise ValueError("Number of Antennas on both side should be same.")
        return y, h, n, U, S, Vh


# **SISO with Rayleigh Channel**

In [77]:
SNRdB = np.linspace(0, 40, 20)
SNR = np.array([10**(ii/10) for ii in SNRdB])
std = np.sqrt(SNR)

# SISO with simple only AWGN channel
ber_awgn = []
for sd in std:
    a = _16QAM()
    s = a.generateSymbol()
    channel = AWGNChannel(len(s))

    # Modulation
    signal, xRG, xIG = a.getSignal(s)

    # Passing through channel
    y, h, n = channel.addChannel(signal, sd)

    # Demodulation
    yR, yI = a.demodulateSignal(y)
    ber_awgn.append(a.getBER(yR, yI, xRG, xIG))


# SISO with Rayleigh channel
ber_siso_rayleigh = []
for sd in std:
    a = _16QAM()
    s = a.generateSymbol()
    channel = RayleighChannel(len(s))

    # Modulation
    signal, xRG, xIG = a.getSignal(s)
    _y, h, n = channel.addChannel(signal, sd)

    # Equalization
    y = _y/h

    # Demodulation
    yR, yI = a.demodulateSignal(y)
    ber_siso_rayleigh.append(a.getBER(yR, yI, xRG, xIG))

In [81]:
# Plotting BERs
plt.semilogy(SNRdB, ber_awgn, "-o", label="SISO AWGN")
plt.semilogy(SNRdB, ber_siso_rayleigh, "-o", label="SISO Rayleigh")
plt.grid(True)
plt.xlim(0, 17)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(10**(-3),10**0)
plt.xlabel("SNR in dB", fontsize=20)
plt.ylabel("BER", fontsize=20)
plt.title("16-QAM BER vs SNR Plot", fontsize=20)
plt.legend(fontsize=11)

## Understanding BER using constellation map

With increase in SNR the symbols are stay closer to the actual symbol therefore BER remains low

In [83]:
# Plotting constellation
sd = [1, 10, 20, 38]

_signals = []
for _ in sd:
    aa = _16QAM()
    ss = aa.generateSymbol()
    _channel = RayleighChannel(len(ss))
    _signal, _xRG, _xIG = aa.getSignal(ss)
    _yy, hh, nn = _channel.addChannel(_signal, _)

    # Equalization
    yy = _yy/hh
    re = np.real(np.sqrt(10)*yy[:1600])
    im = np.imag(np.sqrt(10)*yy[:1600])
    _signals.append([re, im])

for i in range(4):
    plt.subplot(2, 2, i+1)
    plt.scatter(_signals[i][0][:600], _signals[i][1][:600])
    plt.scatter(np.real(np.sqrt(10)*_signal[:1000]), np.imag(np.sqrt(10)*_signal[:1000]), color="red")
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlim(-4, 4)
    plt.ylim(-4, 4)
    plt.grid()
    plt.xlabel("In-phase Component", fontsize=10)
    plt.ylabel("Quadrature Component", fontsize=10)
    plt.title("16-QAM Constellation", fontsize=10)
    plt.tight_layout()

# **SIMO with Rayleigh Channel**

In [84]:
SNRdB = np.linspace(0, 40, 20)
SNR = np.array([10**(ii/10) for ii in SNRdB])
std = np.sqrt(SNR)

# SISO with simple only AWGN channel
ber_awgn = []
for sd in std:
    a = _16QAM()
    s = a.generateSymbol()
    channel = AWGNChannel(len(s))

    # Modulation
    signal, xRG, xIG = a.getSignal(s)

    # Passing through channel
    y, h, n = channel.addChannel(signal, sd)

    # Demodulation
    yR, yI = a.demodulateSignal(y)
    ber_awgn.append(a.getBER(yR, yI, xRG, xIG))


# SISO with Rayleigh channel
ber_siso_rayleigh = []
for sd in std:
    a = _16QAM()
    s = a.generateSymbol()
    channel = RayleighChannel(len(s))

    # Modulation
    signal, xRG, xIG = a.getSignal(s)
    _y, h, n = channel.addChannel(signal, sd)

    # Equalization
    y = _y/h

    # Demodulation
    yR, yI = a.demodulateSignal(y)
    ber_siso_rayleigh.append(a.getBER(yR, yI, xRG, xIG))


# SIMO with Rayleigh channel having selection combining (SC) at receiver
ber_simo_sc = []
for sd in std:
    a = _16QAM()
    s = a.generateSymbol()
    channel = RXDiversityChannel(len(s), numReceiveAntenna=4)

    # Modulation
    signal, xRG, xIG = a.getSignal(s)
    r, h, n = channel.addChannel(signal, sd)

    # Selection Combining
    _max = np.argmax(np.absolute(h), axis=1)                                
    h_max_indxs = tuple(np.append(np.arange(len(s))[:, None], _max, axis=1).T)  
    r_max_indxs = tuple(np.append(np.arange(len(s))[:, None], _max, axis=1).T)
    
    # Equalization
    y = r.squeeze()[r_max_indxs] / h.squeeze()[h_max_indxs]
    
    # Demodulation
    yR, yI = a.demodulateSignal(y)
    ber_simo_sc.append(a.getBER(yR, yI, xRG, xIG))


# SIMO with Rayleigh channel having maximum ratio ccombining (MRC) at receiver
ber_simo_mrc = []
for sd in std:
    a = _16QAM()
    s = a.generateSymbol()
    channel = RXDiversityChannel(len(s), numReceiveAntenna=4)
    
    # Modulation
    signal, xRG, xIG = a.getSignal(s)
    r, h, n = channel.addChannel(signal, sd)

    # MRC 
    _h = np.matmul(np.linalg.inv(np.matmul(np.transpose(h.conj(),(0, 2, 1)), h)), np.transpose(h.conj(),(0, 2, 1))) # finds Hh/norm(H), here Hh is hermition(H)
    
    # Equalization 
    y = np.matmul(_h, r).squeeze()                                              # y = (Hh/norm(H)).y
    
    # Demodulation
    yR, yI = a.demodulateSignal(y)
    ber_simo_mrc.append(a.getBER(yR, yI, xRG, xIG))


In [86]:
# Plotting BERs
plt.semilogy(SNRdB, ber_awgn, "-o", label="SISO AWGN")
plt.semilogy(SNRdB, ber_siso_rayleigh, "-o", label="SISO Rayleigh")
plt.semilogy(SNRdB, ber_simo_sc, "-o", label="SIMO_SC Rayleigh")
plt.semilogy(SNRdB, ber_simo_mrc, "-o", label="SIMO_MRC Rayleigh")
plt.grid(True)
plt.xlim(0, 17)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(10**(-3),10**0)
plt.xlabel("SNR in dB", fontsize=20)
plt.ylabel("BER", fontsize=20)
plt.title("16-QAM BER vs SNR Plot", fontsize=20)
plt.legend(fontsize=11)

# **SIMO Exercise**

Run SIMO with MRC with Rayleigh Channel with antenna configurations as 1x2, 1x4, 1x6, and 1x8. Plot BERs for each case and point out the observations

In [89]:
SNRdB = np.linspace(0, 40, 20)
SNR = np.array([10**(ii/10) for ii in SNRdB])
std = np.sqrt(SNR)

# SIMO with Rayleigh channel having maximum ratio ccombining (MRC) at receiver
ber_simo_mrc_antn = []
for antenna in range(2, 9, 2):
    tmp_ber = []
    for sd in std:
        a = _16QAM()
        s = a.generateSymbol()

        # Add one line here to create channel
        
        
        # Modulation
        signal, xRG, xIG = a.getSignal(s)
        r, h, n = channel.addChannel(signal, sd)

        # MRC 
        _h = np.matmul(np.linalg.inv(np.matmul(np.transpose(h.conj(),(0, 2, 1)), h)), np.transpose(h.conj(),(0, 2, 1))) # finds Hh/norm(H), here Hh is hermition(H)
        
        # Equalization 
        y = np.matmul(_h, r).squeeze()                                              # y = (Hh/norm(H)).y
        
        # Demodulation
        yR, yI = a.demodulateSignal(y)
        tmp_ber.append(a.getBER(yR, yI, xRG, xIG))
    ber_simo_mrc_antn.append(tmp_ber)


In [95]:
# Plotting BERs
antn = [2, 4, 6, 8]
for i in range(4):
    plt.semilogy(SNRdB, ber_simo_mrc_antn[i], "-o", label=f"SIMO_MRC {antn[i]}x{antn[i]}")
plt.grid(True)
plt.xlim(0, 17)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(10**(-3),10**0)
plt.xlabel("SNR in dB", fontsize=20)
plt.ylabel("BER", fontsize=20)
plt.title("16-QAM BER vs SNR Plot", fontsize=20)
plt.legend(fontsize=11)

# **MIMO with Rayleigh Channel**

In [3]:
SNRdB = np.linspace(0, 40, 20)
SNR = np.array([10**(ii/10) for ii in SNRdB])
std = np.sqrt(SNR)

# MIMO with Rayleigh channel having Zero Forcing (ZF) receiver
ber_mimo_zf = []
capacity = []
for sd in std:
    a = _16QAM()
    s = a.generateSymbol()
    channel = MIMOChannel(len(s), numReceiveAntenna=4, numTransmitAntenna=4)
    signal, xRG, xIG = a.getSignal(s)
    y, h, n, U, S, Vh = channel.addChannel(signal, sd, technique="zf")          # U, S, Vh is None no need is ZF just for consistency with svd function
    
    # ZF
    _h = np.matmul(np.linalg.inv(np.matmul(np.transpose(h.conj(),(0, 2, 1)), h)), np.transpose(h.conj(),(0, 2, 1))) # _h = inv(HhH)Hh

    # Equalization
    _y = np.matmul(_h, y)                                                       # inv(HhH)Hhy

    # Demodulation
    yR, yI = a.demodulateSignal(_y.squeeze())
    ber_mimo_zf.append(a.getBER(yR.flatten(), yI.flatten(), xRG, xIG))


# MIMO with Rayleigh channel with SVD
ber_mimo_svd = []
capacity = []
for sd in std:
    a = _16QAM()
    s = a.generateSymbol()
    channel = MIMOChannel(len(s), numReceiveAntenna=4, numTransmitAntenna=4)
    signal, xRG, xIG = a.getSignal(s)
    y, h, n, U, S, Vh = channel.addChannel(signal, sd, technique="svd")
    
    # Equalization or Decoding precoded signal
    decoded_y = np.matmul(np.transpose(U.conj(), (0, 2, 1)), y)/S                # y = Uhy 

    # Demodulation
    yR, yI = a.demodulateSignal(decoded_y.squeeze())
    ber_mimo_svd.append(a.getBER(yR.flatten(), yI.flatten(), xRG, xIG))

In [17]:
# Plotting BERs
plt.semilogy(SNRdB, ber_awgn, "-o", label="SISO AWGN")
plt.semilogy(SNRdB, ber_siso_rayleigh, "-o", label="SISO Rayleigh")
plt.semilogy(SNRdB, ber_simo_sc, "-o", label="SIMO_SC Rayleigh")
plt.semilogy(SNRdB, ber_simo_mrc, "-o", label="SIMO_MRC Rayleigh")
plt.semilogy(SNRdB, ber_mimo_zf, "k", label="MIMO_ZF Rayleigh")
plt.semilogy(SNRdB, ber_mimo_svd, "-o", label="MIMO_SVD Rayleigh")
plt.grid(True)
plt.xlim(0, 17)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(10**(-3),10**0)
plt.xlabel("SNR in dB", fontsize=20)
plt.ylabel("BER", fontsize=20)
plt.title("16-QAM BER vs SNR Plot", fontsize=20)
plt.legend(fontsize=11)

# Comparing All Schemes for a single SNR value

In [10]:
# Plotting constellation
sd = 10

_conste = []

# SISO with simple only AWGN channel
a = _16QAM()
s = a.generateSymbol()
channel = AWGNChannel(len(s))
# Modulation
signal, xRG, xIG = a.getSignal(s)
# Passing through channel
y, h, n = channel.addChannel(signal, sd)
re = np.real(np.sqrt(10)*y[:1600])
im = np.imag(np.sqrt(10)*y[:1600])
_conste.append([re, im])


# SISO Rayleigh
aa = _16QAM()
ss = aa.generateSymbol()
_channel = RayleighChannel(len(ss))
_signal, _xRG, _xIG = aa.getSignal(ss)
_yy, hh, nn = _channel.addChannel(_signal, sd)
# Equalization
yy = _yy/hh
re = np.real(np.sqrt(10)*yy[:1600])
im = np.imag(np.sqrt(10)*yy[:1600])
_conste.append([re, im])


# SIMO SC
a = _16QAM()
s = a.generateSymbol()
channel = RXDiversityChannel(len(s), numReceiveAntenna=4)
# Modulation
signal, xRG, xIG = a.getSignal(s)
r, h, n = channel.addChannel(signal, sd)
# Selection Combining
_max = np.argmax(np.absolute(h), axis=1)                                
h_max_indxs = tuple(np.append(np.arange(len(s))[:, None], _max, axis=1).T)  
r_max_indxs = tuple(np.append(np.arange(len(s))[:, None], _max, axis=1).T)
# Equalization
y = r.squeeze()[r_max_indxs] / h.squeeze()[h_max_indxs]
re = np.real(np.sqrt(10)*y[:1600])
im = np.imag(np.sqrt(10)*y[:1600])
_conste.append([re, im])

# SIMO MRC
a = _16QAM()
s = a.generateSymbol()
channel = RXDiversityChannel(len(s), numReceiveAntenna=4)
# Modulation
signal, xRG, xIG = a.getSignal(s)
r, h, n = channel.addChannel(signal, sd)
# MRC 
_h = np.matmul(np.linalg.inv(np.matmul(np.transpose(h.conj(),(0, 2, 1)), h)), np.transpose(h.conj(),(0, 2, 1))) # finds Hh/norm(H), here Hh is hermition(H)
# Equalization 
y = np.matmul(_h, r).squeeze()                                              # y = (Hh/norm(H)).y
re = np.real(np.sqrt(10)*y[:1600])
im = np.imag(np.sqrt(10)*y[:1600])
_conste.append([re, im])


# ZF
a = _16QAM()
s = a.generateSymbol()
channel = MIMOChannel(len(s), numReceiveAntenna=4, numTransmitAntenna=4)
signal, xRG, xIG = a.getSignal(s)
y, h, n, U, S, Vh = channel.addChannel(signal, sd, technique="zf")          # U, S, Vh is None no need is ZF just for consistency with svd function
_h = np.matmul(np.linalg.inv(np.matmul(np.transpose(h.conj(),(0, 2, 1)), h)), np.transpose(h.conj(),(0, 2, 1))) # _h = inv(HhH)Hh
# Equalization
_y = np.matmul(_h, y)                                                       # inv(HhH)Hhy
re = np.real(np.sqrt(10)*_y[:1600])
im = np.imag(np.sqrt(10)*_y[:1600])
_conste.append([re, im])


# SVD
a = _16QAM()
s = a.generateSymbol()
channel = MIMOChannel(len(s), numReceiveAntenna=4, numTransmitAntenna=4)
signal, xRG, xIG = a.getSignal(s)
y, h, n, U, S, Vh = channel.addChannel(signal, sd, technique="svd")
# Equalization or Decoding precoded signal
decoded_y = np.matmul(np.transpose(U.conj(), (0, 2, 1)), y)/S                # y = Uhy 
re = np.real(np.sqrt(10)*decoded_y[:1600])
im = np.imag(np.sqrt(10)*decoded_y[:1600])
_conste.append([re, im])

In [19]:
name = ["SISO_AWGN", "SISO_Rayleigh", "SIMO_SC", "SIMO_MRC", "MIMO_ZF", "MIMO_SVD"]
plt.figure(figsize=(10, 8))
for i in range(6):
    plt.subplot(3, 3, i+1)
    plt.scatter(_conste[i][0][:600], _conste[i][1][:600])
    plt.scatter(np.real(np.sqrt(10)*signal[:1000]), np.imag(np.sqrt(10)*signal[:1000]), color="red")
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlim(-4, 4)
    plt.ylim(-4, 4)
    plt.grid()
    plt.xlabel("In-phase Component", fontsize=10)
    plt.ylabel("Quadrature Component", fontsize=10)
    plt.title(name[i], fontsize=10)
    plt.tight_layout()